[FROM HERe see above](https://www.makeuseof.com/working-with-toml-files-in-rust/)

In [4]:
:dep serde = { version = "1.0", features = ["derive"] }
:dep toml = "0.7.6"

In [5]:
use std::fs::File;
use std::io::Write;
use serde::Serialize;
use toml::to_string;

#[derive(Serialize)]
struct ServerConfig {
    host: String,
    port: u16,
    timeout: u32,
}

fn write_config_to_file(config: &ServerConfig, file_path: &str) -> Result<(), Box<dyn std::error::Error>> {
    let toml_string = to_string(config)?;
    let mut file = File::create(file_path)?;
    file.write_all(toml_string.as_bytes())?;
    Ok(())
}

fn main_to_file() {
    let config = ServerConfig {
        host: "localhost".to_owned(),
        port: 8000,
        timeout: 30,
    };

    if let Err(e) = write_config_to_file(&config, "config.toml") {
        eprintln!("Error: {}", e);
    } else {
        println!("Config file created successfully.");
    }
}
        
 

In [7]:
main_to_file();

Config file created successfully.


In [19]:
use std::fs::File;
use std::io::Write;
use serde::Serialize;
use toml::to_string;

#[derive(Serialize)]
struct ServerConfig {
    host: String,
    port: u16,
    timeout: u32,
}

#[derive(Serialize)]
struct Screener {
    url_for_screener: String
}

fn write_config_to_file(config: &ServerConfig, file_path: &str) -> Result<(), Box<dyn std::error::Error>> {
    let toml_string = to_string(config)?;
    let mut file = File::create(file_path)?;
    file.write_all(toml_string.as_bytes())?;
    Ok(())
}

fn write_screener_to_file(screener: &Screener, file_path: &str) -> Result<(), Box<dyn std::error::Error>> {
    let toml_string = to_string(screener)?;
    let mut file = File::create(file_path)?;
    file.write_all(toml_string.as_bytes())?;
    Ok(())
}

fn main_to_file_two() {
    let screener = Screener{
        url_for_screener: "localhost".to_string(),
        };

    if let Err(e) = write_screener_to_file(&screener, "screener.toml") {
        eprintln!("Error: {}", e);
    } else {
        println!("Screener file created successfully.");
    }
}
        
 

In [23]:
main_to_file_two();

Screener file created successfully.


In [21]:
use std::collections::HashMap;
use serde::{Deserialize, Serialize};

#[derive(Debug, Serialize, Deserialize)]
struct Config {
    flag1: bool,
    flag2: bool,
    // the nested table should go last to prevent the `toml::ser::Error::ValueAfterTable`
    #[serde(with = "sentinel_to_none")]
    values: HashMap<String, Option<u32>>,
}

mod sentinel_to_none {
    use super::*;
    use serde::{Deserializer, Serializer};

    const SENTINEL: u32 = 0;

    type SerdeType = HashMap<String, u32>;
    type DesiredType = HashMap<String, Option<u32>>;

    pub fn serialize<S>(value: &DesiredType, serializer: S) -> Result<S::Ok, S::Error>
    where
        S: Serializer,
    {
        let with_sentinel: SerdeType = value
            .clone()
            .into_iter()
            .map(|(name, optional)| {
                let flat = optional.unwrap_or(SENTINEL);
                (name, flat)
            })
            .collect();

        with_sentinel.serialize(serializer)
    }

    pub fn deserialize<'de, D>(deserializer: D) -> Result<DesiredType, D::Error>
    where
        D: Deserializer<'de>,
    {
        let with_sentinel = SerdeType::deserialize(deserializer)?;
        let with_options = with_sentinel
            .into_iter()
            .map(|(name, flat)| {
                let optional = if flat == SENTINEL { None } else { Some(flat) };
                (name, optional)
            })
            .collect();

        Ok(with_options)
    }
}

fn main() {
    let values = [("foo", Some(5)), ("bar", None)]
        .iter()
        .map(|(name, s)| (name.to_string(), s.clone()))
        .collect();

    let config = Config {
        flag1: true,
        flag2: false,
        values,
    };

    let s = toml::ser::to_string(&config).unwrap();
    println!("Serialized config: \n{}", s);

    println!("-------------------------------\n");

    let s = "flag1 = false
        flag2 = true
        
        [values]
        should_be_none = 0
        should_be_some = 42";
    let config_deser: Config = toml::de::from_str(s).unwrap();
    println!("Deserialized: {:#?}", config_deser);
}


In [22]:
main();

Serialized config: 
flag1 = true
flag2 = false

[values]
foo = 5
bar = 0

-------------------------------

Deserialized: Config {
    flag1: false,
    flag2: true,
    values: {


        "should_be_some": Some(
            42,
        ),
        "should_be_none": None,
    },
}
